# Stochastic simulation of fractured volumes

In this totorial, we provide simple stochastic simulation functions to create fractured volumes.
This is an ongoing work aiming at creating BReps standing for:
 * multiple fractured layers
 * 3D fractured granitic rock

These workflows also uses functionalities of [Geode-Explicit](https://docs.geode-solutions.com/references/geode-explicit/).

In [ ]:
import opengeode as og 
import opengeode_io 
import opengeode_inspector as inspector
import geode_explicit as explicit
import geode_common as common
import geode_simplex as simplex
import geode_conversion as conversion


print("Geode module imported!")

# define various utilities functions
import os
def input(path):
  return os.path.join("data/",path) 

def create_folder_if_does_not_exists(path):
  if( not os.path.isdir(path)):
    os.makedirs(path)

def output(path):
  result_path = "results/"
  create_folder_if_does_not_exists(result_path)
  return os.path.join(result_path,path)

# define print section description
def print_section_components(section) : 
    bb =og.BoundingBox2D()
    bb.add_box(section.bounding_box())
    print("The section is centered on (" + 
          str(bb.center().value(0))+" , "+str(bb.center().value(1)) + 
          "). It is a "+str(bb.diagonal().value(0))+"m by "+
          str(bb.diagonal().value(1))+
          "m section with: ")
    print(" * "+ str(section.nb_corners()) +" corners" )
    for corner in section.corners():
        print("  - corner "+ corner.id().string()+" have " + str(corner.mesh().nb_vertices())+" vertices.") 
    print(" * "+ str(section.nb_lines()) +" lines" )
    for line in section.lines():
        print("  - line "+ line.id().string()+" have " + str(line.mesh().nb_edges())+" edges.") 
    print(" * "+ str(section.nb_surfaces()) +" surfaces" )
    for surface in section.surfaces():
        print("  - surface "+ surface.id().string()+" have " + str(surface.mesh().nb_polygons())+" triangles.") 

# define print section description
def print_brep_components(brep) : 
    bb =og.BoundingBox3D()
    bb.add_box(brep.bounding_box())
    print("The BRep is centered on (" + 
          str(bb.center().value(0))+" , "+str(bb.center().value(1)) + 
          "). It is a "+str(bb.diagonal().value(0))+"m by "+
          str(bb.diagonal().value(1))+
          "m BRep with: ")
    print(" * "+ str(brep.nb_corners()) +" corners" )
    for corner in brep.corners():
        print("  - corner "+ corner.id().string()+" have " + str(corner.mesh().nb_vertices())+" vertices.") 
    print(" * "+ str(brep.nb_lines()) +" lines" )
    for line in brep.lines():
        print("  - line "+ line.id().string()+" have " + str(line.mesh().nb_edges())+" edges.") 
    print(" * "+ str(brep.nb_surfaces()) +" surfaces" )
    for surface in brep.surfaces():
        print("  - surface "+ surface.id().string()+" have " + str(surface.mesh().nb_polygons())+" triangles.") 
    print(" * "+ str(brep.nb_blocks()) +" blocks" )
    for block in brep.blocks():
        print("  - block "+ block.id().string()+" have " + str(block.mesh().nb_polyhedra())+" polyhedra.") 

def create_aligned_box2D(extent_x,extent_y):
    bbox = og.BoundingBox2D()
    bbox.add_point(og.Point2D([0,0]))
    bbox.add_point(og.Point2D([extent_x,extent_y]))
    return bbox

def create_aligned_box3D(extent_x,extent_y,extent_z):
    bbox = og.BoundingBox3D()
    bbox.add_point(og.Point3D([0,0,0]))
    bbox.add_point(og.Point3D([extent_x,extent_y,extent_z]))
    return bbox

def create_edged_curve(points):
    curve = og.EdgedCurve2D.create()
    builder = og.EdgedCurveBuilder2D.create(curve)
    v0 = builder.create_point(points[0])
    for point in points[1::]:
        v1 = builder.create_point(point)
        builder.create_edge_with_vertices(v0,v1)
        v0 = v1
    return curve

import random 
import math

# random selection of a point in 2D Bounding Box
def draw_point_in_box(bbox):
    randx = random.uniform(bbox.min().value(0),bbox.max().value(0))
    randy =random.uniform(bbox.min().value(1),bbox.max().value(1))
    return og.Point2D([randx,randy])

# random selection of a point in 2D ring
def draw_point_in_radial_shape(center, rmin,rmax):
    r = rmin + ( rmax - rmin ) * math.sqrt( random.random() )
    theta = ( 2.0 * math.pi ) * random.random()
    randx = center.value( 0 ) + r * math.cos( theta )
    randy =center.value( 1 ) + r * math.sin( theta )
    return og.Point2D([randx,randy])
    
# create one fracture trace
def create_one_fracture_trace(bbox,min_length,max_length):
    tips0= draw_point_in_box(bbox)
    tips1= draw_point_in_radial_shape(tips0,min_length,max_length)
    points= [tips0,tips1]
    return create_edged_curve(points)

def create_2Dbox_section(box):
    c0 = box.min()
    c1= og.Point2D([box.min().value(0),box.max().value(1)])
    c2 =  box.max()
    c3= og.Point2D([box.max().value(0),box.min().value(1)])
    explicit_modeler = explicit.SectionExplicitModeler()
    ec1 = create_edged_curve([c0,c1])
    ec2 = create_edged_curve([c1,c2])
    ec3 = create_edged_curve([c2,c3])
    ec4 = create_edged_curve([c3,c0])
    explicit_modeler.add_curve(ec1)
    explicit_modeler.add_curve(ec2)
    explicit_modeler.add_curve(ec3)
    explicit_modeler.add_curve(ec4)
    return explicit_modeler.build()

def create_fractures(number,box,min_length,max_length):
    _fractures=[]
    for _ in range(number):
        _fractures.append(create_one_fracture_trace(box,min_length,max_length))
    return _fractures 

def remesh_and_extrude_section(section, cst_cell_size, ztop, zbot):
   #remesh
   metric = common.ConstantMetric2D(cst_cell_size)
   remeshed_sec = simplex.section_simplex_remesh(section,metric)[0]
   #extrude
   ext_opt = og.SectionExtruderOptions()
   ext_opt.axis_to_extrude = 2
   ext_opt.min_coordinate=ztop
   ext_opt.max_coordinate=zbot
   brep = og.extrude_section_to_brep(remeshed_sec, ext_opt)
   return brep

def create_fractured_layer(box, nb_fractures, min_length, max_length, ztop, zbot):
    explicit_modeler = explicit.SectionExplicitModeler()
    boundaries = create_2Dbox_section(box)
    explicit_modeler.add_section(boundaries)
    fracture_network = create_fractures(nb_fractures,box,min_length,max_length)
    for fracture in fracture_network:
        explicit_modeler.add_curve(fracture)
    section = explicit_modeler.build()
    return remesh_and_extrude_section(section,min_length,ztop,zbot)

## Create a multi-layered BRep
This part creates several fractured layers and merge them in a unique BRep.

In [ ]:
box_extent = 10
box2D = create_aligned_box2D(box_extent,box_extent)

nb_fractures = 10
min_length = 1
max_length = 5
ztop = 0
zbot = -(0.25*box_extent)
brep0 = create_fractured_layer(box2D, nb_fractures, min_length, max_length, ztop, zbot)
og.triangulate_brep_surface_meshes(brep0)
og.convert_brep_surface_meshes_into_triangulated_surfaces(brep0)
og.save_brep(brep0,  output("fractured_brep0.vtm"))

nb_fractures = 20
min_length = 1
max_length = 5
ztop = zbot
zbot = zbot + zbot
brep1 = create_fractured_layer(box2D, nb_fractures, min_length, max_length, ztop, zbot)
og.triangulate_brep_surface_meshes(brep1)
og.convert_brep_surface_meshes_into_triangulated_surfaces(brep1)
og.save_brep(brep1,  output("fractured_brep1.vtm"))

box3D = create_aligned_box3D(box_extent,box_extent,zbot)
brep_explicit_modeler = explicit.BRepExplicitModeler()
brep_explicit_modeler.add_brep(brep0)
brep_explicit_modeler.add_brep(brep1)
merged_brep = brep_explicit_modeler.build()

og.save_brep(merged_brep,  output("fractured_merged_brep.og_brep"))
og.save_brep(merged_brep,  output("fractured_merged_brep2layer.vtm"))


In [ ]:
# remesh
# set up parameters for homogeneous remeshing
edge_length = 1
cst_metric = common.ConstantMetric3D(edge_length)

# create and return the remeshed section
remeshed_rand_brep = simplex.brep_simplex_remesh(merged_brep,cst_metric)[0]

og.save_brep(remeshed_rand_brep,output("remeshed_merged_brep.vtm"))
print_brep_components(remeshed_rand_brep)

## Create BRep with random planar surfaces

In [ ]:
import opengeode as og 
import opengeode_io 
import opengeode_inspector as inspector
import geode_explicit as explicit
import geode_common as common
import geode_simplex as simplex

print("Geode module imported!")
# define various utilities functions
import os
def input(path):
  return os.path.join("data/",path) 

def create_folder_if_does_not_exists(path):
  if( not os.path.isdir(path)):
    os.makedirs(path)

def output(path):
  result_path = "results/"
  create_folder_if_does_not_exists(result_path)
  return os.path.join(result_path,path)

# define print section description
def print_section_components(section) : 
    bb =og.BoundingBox2D()
    bb.add_box(section.bounding_box())
    print("The section is centered on (" + 
          str(bb.center().value(0))+" , "+str(bb.center().value(1)) + 
          "). It is a "+str(bb.diagonal().value(0))+"m by "+
          str(bb.diagonal().value(1))+
          "m section with: ")
    print(" * "+ str(section.nb_corners()) +" corners" )
    for corner in section.corners():
        print("  - corner "+ corner.id().string()+" have " + str(corner.mesh().nb_vertices())+" vertices.") 
    print(" * "+ str(section.nb_lines()) +" lines" )
    for line in section.lines():
        print("  - line "+ line.id().string()+" have " + str(line.mesh().nb_edges())+" edges.") 
    print(" * "+ str(section.nb_surfaces()) +" surfaces" )
    for surface in section.surfaces():
        print("  - surface "+ surface.id().string()+" have " + str(surface.mesh().nb_polygons())+" triangles.") 

# define print section description
def print_brep_components(brep) : 
    bb =og.BoundingBox3D()
    bb.add_box(brep.bounding_box())
    print("The BRep is centered on (" + 
          str(bb.center().value(0))+" , "+str(bb.center().value(1)) + 
          "). It is a "+str(bb.diagonal().value(0))+"m by "+
          str(bb.diagonal().value(1))+
          "m BRep with: ")
    print(" * "+ str(brep.nb_corners()) +" corners" )
    for corner in brep.corners():
        print("  - corner "+ corner.id().string()+" have " + str(corner.mesh().nb_vertices())+" vertices.") 
    print(" * "+ str(brep.nb_lines()) +" lines" )
    for line in brep.lines():
        print("  - line "+ line.id().string()+" have " + str(line.mesh().nb_edges())+" edges.") 
    print(" * "+ str(brep.nb_surfaces()) +" surfaces" )
    for surface in brep.surfaces():
        print("  - surface "+ surface.id().string()+" have " + str(surface.mesh().nb_polygons())+" triangles.") 
    print(" * "+ str(brep.nb_blocks()) +" blocks" )
    for block in brep.blocks():
        print("  - block "+ block.id().string()+" have " + str(block.mesh().nb_polyhedra())+" tetrahedra.") 

def create_aligned_box3D(extent):
    bbox = og.BoundingBox3D()
    bbox.add_point(og.Point3D([0,0,0]))
    bbox.add_point(og.Point3D([extent,extent,extent]))
    return bbox

# create surface from its orbital points in 3D
def create_surface_from_orbital_points(points):
    surface = og.PolygonalSurface3D.create()
    builder = og.PolygonalSurfaceBuilder3D.create(surface)
    vertices=[]
    for point in points:
        vertices.append(builder.create_point(point))
    builder.create_polygon(vertices)
    og.triangulate_surface_mesh3D(surface)
    return og.convert_surface_mesh_into_triangulated_surface3D(surface)

def create_aligned_boxBRep3D(box):
    # corner points on the top of the box. 
    top1 = og.Point3D([box.min().value(0),box.min().value(1),box.max().value(2)])
    top2 = og.Point3D([box.min().value(0),box.max().value(1),box.max().value(2)])
    top3 = og.Point3D([box.max().value(0),box.max().value(1),box.max().value(2)])
    top4 = og.Point3D([box.max().value(0),box.min().value(1),box.max().value(2)])
    # corner points on the bottom of the box. 
    bot1 = og.Point3D([box.min().value(0),box.min().value(1),box.min().value(2)])
    bot2 = og.Point3D([box.min().value(0),box.max().value(1),box.min().value(2)])
    bot3 = og.Point3D([box.max().value(0),box.max().value(1),box.min().value(2)])
    bot4 = og.Point3D([box.max().value(0),box.min().value(1),box.min().value(2)])
    # surface of the bounding box. 
    top = create_surface_from_orbital_points([top1,top2,top3,top4])
    bottom = create_surface_from_orbital_points([bot1,bot2,bot3,bot4])
    front = create_surface_from_orbital_points([top1,top4,bot4,bot1])
    back = create_surface_from_orbital_points([top2,top3,bot3,bot2])
    left = create_surface_from_orbital_points([top1,top2,bot2,bot1])
    right = create_surface_from_orbital_points([top3,top4,bot4,bot3])
    # explicit modeling to create the BRep
    modeler = explicit.BRepExplicitModeler()
    modeler.add_triangulated_surface(top)
    modeler.add_triangulated_surface(bottom)
    modeler.add_triangulated_surface(front)
    modeler.add_triangulated_surface(back)
    modeler.add_triangulated_surface(left)
    modeler.add_triangulated_surface(right)
    return modeler.build()

### Add random surfaces

To create a triangulated surface easily, we compte vertices defining corners of a square from four parameters:
1. the vertex at the center,
1. the azimuth angle measured clockwise from the North (y axis) in the xy plane,
1. the dip angle measured clockwise from the x axis in the xz plane,
1. the extent of the square 

In [ ]:
import math
# compute vertices to define a square 
def compute_orbital_points(center,azimuth_deg,dip_deg,length,nb_points):

    deg_to_rad = math.pi/180.
    dip_rad = dip_deg * deg_to_rad
    az_rad = azimuth_deg * deg_to_rad
    az_dir = og.Vector3D(og.Point3D([0,0,0]),og.Point3D([math.sin(az_rad),math.cos(az_rad),0]))
    dip_dir = og.Vector3D(og.Point3D([0,0,0]),og.Point3D([math.cos(dip_rad)*math.cos(az_rad),-math.cos(dip_rad)*math.sin(az_rad),-math.sin(dip_rad)]))
    half_length = 0.5*length

    points=[]
    for point in range(nb_points):
        agl = point*2*math.pi/nb_points
        points.append(center + 
                      az_dir* half_length *  math.cos(agl)  +
                      dip_dir * half_length * math.sin(agl) )
    return points

# create a squared triangulated surface from center, orientations and size
def create_triangulated_surface(center,azimuth_deg,dip_deg,length):
    orbital_points = compute_orbital_points(center,azimuth_deg,dip_deg,length,10)
    return create_surface_from_orbital_points(orbital_points)

# create a surface to vizualyse in paraview.
surface = create_triangulated_surface(og.Point3D([5,5,10]), 10, 10, 10 )
og.save_triangulated_surface3D(surface,output("surface.vtp"))

In [ ]:
import random 
import math

# random selection of a point in 2D Bounding Box
def draw_point_in_box3d(box):
    randx = random.uniform(box.min().value(0),box.max().value(0))
    randy = random.uniform(box.min().value(1),box.max().value(1))
    randz = random.uniform(box.min().value(2),box.max().value(2))
    return og.Point3D([randx,randy,randz])

def create_random_surfaces(box):
    center = draw_point_in_box3d(box)
    azimuth = random.uniform(0.,180.)
    dip = random.uniform(0.,90.)
    box_diag_length = box.diagonal().length()
    length = random.uniform(0.2*box_diag_length, 0.5*box_diag_length)
    return create_triangulated_surface(center,azimuth,dip,length)

box_extent = 10
nb_fractures = 10

box = create_aligned_box3D(box_extent)
brep_box = create_aligned_boxBRep3D(box)
modeler = explicit.BRepExplicitModeler()
modeler.add_brep(brep_box)
for i in range(nb_fractures):
    modeler.add_triangulated_surface(create_random_surfaces(box))
rand_brep = modeler.build()
og.save_brep(rand_brep,output("rand_brep.vtm"))

# inspector
brep_inspector = inspector.BRepInspector(rand_brep)
result = brep_inspector.inspect_brep()
print(result.string())

In [ ]:
# set up parameters for homogeneous remeshing
edge_length = 1
cst_metric = common.ConstantMetric3D(edge_length)

# create and return the remeshed section
remeshed_rand_brep = simplex.brep_simplex_remesh(rand_brep,cst_metric)[0]

og.save_brep(remeshed_rand_brep,output("remeshed_rand_brep.vtm"))
print_brep_components(remeshed_rand_brep)